In [1]:
import pandas as pd
import numpy as np

from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# Parameter setting
np.random.seed(2018)
model_name = 'VGG16'
img_size = (224, 224)
batch_size = 32
freeze_layer_num = 10

In [3]:
# Data preparation
train_generator = ImageDataGenerator(preprocessing_function = vgg16.preprocess_input,
                                        rotation_range=10,
                                        width_shift_range=0.05*2,
                                        height_shift_range=0.05*2,
                                        shear_range=0.1*2,
                                        zoom_range=0.1*2,)
valid_test_gen = ImageDataGenerator(preprocessing_function = vgg16.preprocess_input)

train_data_generator = train_generator.flow_from_directory('.\\imgs\\new_train',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          class_mode='categorical')
valid_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\new_valid',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          class_mode='categorical')
test_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\test',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=False,
                                                          class_mode=None)

Found 19972 images belonging to 10 classes.
Found 2452 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [4]:
# Construct model
from keras import backend as K
K.clear_session()

inputs = Input((*img_size, 3))
base_model = VGG16(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print("total layer count {}".format(len(base_model.layers)))

total layer count 19


In [5]:
# Train the FC layer
base_model.trainabel = False
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_data_generator,
                    steps_per_epoch=len(train_data_generator),
                    epochs=1,
                    validation_data=valid_data_generator,
                    validation_steps=len(valid_data_generator))

Epoch 1/1
625/625 [==============================] - 649s 1s/step - loss: 2.0581 - acc: 0.2739 - val_loss: 1.3521 - val_acc: 0.5681


In [6]:
# fine-tune the model
for i, layer in enumerate(model.layers):
    if i <= freeze_layer_num:
        layer.trainable = False
    else:
        layer.trainable = True
    
model.compile(optimizer=RMSprop(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
125/125 [==============================] - 87s 696ms/step - loss: 0.7938 - acc: 0.7438 - val_loss: 0.9433 - val_acc: 0.6925


In [7]:
if not os.path.exists('.\\models'):
    os.mkdir('.\\models')
model.save('models\\'+model_name+'_L'+ str(freeze_layer_num)+'.h5')

In [8]:
# Train model
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
125/125 [==============================] - 87s 699ms/step - loss: 0.5980 - acc: 0.8105 - val_loss: 0.6320 - val_acc: 0.7961


In [9]:
if not os.path.exists('.\\models'):
    os.mkdir('.\\models')
model.save('models\\'+model_name+'_L'+ str(freeze_layer_num)+'.h5')

In [10]:
# Train model
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
125/125 [==============================] - 87s 697ms/step - loss: 0.4548 - acc: 0.8638 - val_loss: 0.6808 - val_acc: 0.7879


In [11]:
# submition file
model = load_model('models\\'+model_name+'_L'+ str(freeze_layer_num)+'.h5')
df = pd.read_csv("submit_example.csv", index_col='img')
preds = model.predict_generator(test_data_generator, steps=len(test_data_generator), verbose=1)
for i, fname in enumerate(test_data_generator.filenames):
    index = fname[fname.rfind('\\')+1:]
    df.loc[index] = preds[i]

df.to_csv('submission/'+model_name+'_L'+ str(freeze_layer_num)+'pred.csv', float_format='%.3f')

2492/2492 [==============================] - 571s 229ms/step
